In [30]:
import re
import yaml
import numpy as np
import pandas as pd
import doctest
from nameparser import HumanName

In [305]:
%load_ext line_profiler

In [287]:
def list_diff(list1, list2):
    '''returns list after taking set difference of two lists
    '''
    return list(set(list1) - set(list2))


def intersect(list1, list2):
    '''returns list after taking set intersection of two lists'''
    return list(set(list1) & set(list2))

In [125]:
HumanName('Bob De La O K')

<HumanName : [
	title: '' 
	first: 'Bob' 
	middle: '' 
	last: 'De La O K' 
	suffix: ''
	nickname: ''
]>

In [356]:
names = pd.DataFrame({'first_name' : ['DE ANDRE', 'BOB D','A RICHARD  A', 'KIM-TOY', 'JO ANN', 'J T', 'ABDUL-AZIZ', 'MARCELLUS. H', 'ERIN  E M', 'J EDGAR', 'GEORGE H W'],
                     'last_name' : ['DE LA O','JONES V', 'LUQUE-ROSALES', 'MC CARTHY', 'O BRIEN', 'SADOWSKY, JR', 'NORWOOD II', 'BURKE JR J', 'VON KONDRAT', 'HOOVER', 'BUSH']})

In [13]:
names

,first_name,last_name
0,DE ANDRE,DE LA O
1,BOB,JONES V
2,A RICHARD A,LUQUE-ROSALES
3,KIM-TOY,MC CARTHY
4,JO ANN,O BRIEN
5,J T,"SADOWSKY, JR"
6,ABDUL-AZIZ,NORWOOD II
7,MARCELLUS,BURKE JR J
8,ERIN E M,VON KONDRAT
9,J EDGAR,HOOVER


In [248]:
def full_strip_name(x):
    '''returns string after
       removing any redundant whitespace or punctuation from string
       >>> full_strip_name("KIM-TOY")
       'KIMTOY'
       >>> full_strip_name("LUQUE-.ROSALES")
       'LUQUEROSALES'
    '''
    x = re.sub(r'[^\w\s]', '', x)
    return ' '.join(x.split())
doctest.run_docstring_examples(full_strip_name, globals())

def basic_strip_name(x):
    '''returns string after
       removing any redundant whitespace or period from string
       >>> basic_strip_name("Mary-Ellen.")
       'Mary-Ellen'
       >>> basic_strip_name("     SADOWSKY,  J.R")
       'SADOWSKY JR'
    '''
    x = re.sub('\s\s+', ' ',
               re.sub(r'^[ \s]*|\.|\,|\s$', '', x))
    return x
doctest.run_docstring_examples(basic_strip_name, globals())

def split_full_names(names, ln='last_name', fn='first_name'):
    '''returns pandas dataframe of last and first name columns
       made from splitting input pandas series of full names
    
       PYTEST
    '''
    # Fill NaN names with ',' for ease of use
    names = names.fillna(",")
    # Fill names that do not contain letter ('-') as ',' for ease of use
    names = names.map(lambda x: x if re.search('[a-zA-Z]', x) else ",")
    # Split names after last comma into list of length = 2
    # Ex: 'Jones, JR, Bob' -> ['Jones, JR', 'Bob']
    names = names.map(lambda x: x.rsplit(',', 1))
    # Turn series of lists into dataframe
    # with last and first name columns
    names = pd.DataFrame(names.values.tolist(),
                         columns=[ln, fn])
    return names

def extract_suffix_name(x, suffixes):
    '''returns the suffix name in a given string
       >>> extract_suffix_name('BURKE SR J', suffixes=['II', 'III', 'IV', 'JR', 'SR', 'I', 'V'])
       ('BURKE J', 'SR')
       >>> extract_suffix_name('NORWOOD II', suffixes=['II', 'III', 'IV', 'JR', 'SR', 'I', 'V'])
       ('NORWOOD', 'II')
       >>> extract_suffix_name('SAUL K JR', suffixes=['II', 'III', 'IV', 'JR', 'SR'])
       ('SAUL K', 'JR')
       >>> extract_suffix_name('JONES V', suffixes=['II', 'III', 'IV', 'JR', 'SR', 'I', 'V'])
       ('JONES', 'V')
       >>> extract_suffix_name('MICHAEL V', suffixes=['II', 'III', 'IV', 'JR', 'SR'])
       ('MICHAEL V', '')
    '''
    # Split input name (x) by spaces and look for suffix in name pieces
    suffix = [w for w in x.split(" ") if w in suffixes]
    # If any suffix found, return it, else return empty string
    assert len(suffix) < 2, 'Too many suffixes found {}'.format(suffix)
    # If suffix is not empty
    if suffix:
        # Remove suffix from name
        x = re.sub('(^{0}\s)|(\s{0}$)|(\s{0}\s)'.format(suffix[0]),
                   ' ', x)
        x = basic_strip_name(x)
        # Remove white space from suffix
        suffix = suffix[0].replace(' ', '')
    else:
        # Fill suffix with empty string
        suffix = ''
        
    # If name is empty after suffix removed
    # EX: first name = JR
    if not x:
        x = suffix    # Re-assign suffix as name
        suffix = ''   # Replace suffix with empty string
    return x, suffix
doctest.run_docstring_examples(extract_suffix_name, globals())

def extract_middle_initial(x, mi_pattern='(\s[A-Z]\s)|(\s[A-Z]$)',
                           not_pattern='^((DE LA)\s[A-Z]($|\s))',
                           avoid_suffixes=[]):
    '''returns the middle initial in a given string
       >>> extract_middle_initial('BURKE SR J')
       ('BURKE SR', 'J')
       >>> extract_middle_initial('DE LA O JR')
       ('DE LA O JR', '')
       >>> extract_middle_initial('SAUL V')
       ('SAUL', 'V')
       >>> extract_middle_initial('BUSH V', avoid_suffixes=['V', 'I'])
       ('BUSH V', '')
       >>> extract_middle_initial('ERIN E M JR')
       ('ERIN M JR', 'E')
       >>> extract_middle_initial('A RICHARD V')
       ('A RICHARD', 'V')
    '''
    # Search input name for middle initial based on regex pattern
    mi = re.search(mi_pattern, x)
    # Assign identified middle initial if it is found
    # And if the name is too short (Ex: A J, J R)
    # Otherwise mi is assigned as an empty string
    mi = mi.group() if mi and len(x) > 3 else ''
    # If middle initial is not empty
    # And name does not have the not patterns (DE LA O)
    # And the identified middle initial is not in the suffixes
    if (mi and
        not ((not_pattern and 
              re.search(not_pattern, x)) or
             mi[-1] in avoid_suffixes)):
        # Remove the middle initial from the name
        x = re.sub(mi_pattern,' ',x)
        # Remove spaces in middle initial
        mi = mi.replace(' ', '')
    else:
        # Assign middle initial as empty string
        mi = ''
    return basic_strip_name(x), mi
doctest.run_docstring_examples(extract_middle_initial, globals())

In [255]:
def split_name(x, suffixes,
               not_pattern='^((DE LA)\s[A-Z]($|\s))',
               mi_pattern='(\s[A-Z]\s)|(\s[A-Z]$)'):
    '''returns list of name components
       the first element is no-spaced name,
       while the last element may contain spaces
       >>> split_name('DE LA O JR', ['II', 'III', 'IV', 'JR', 'SR', 'V', 'I'])
       ['DELAO', '', '', 'JR', 'DE LA O']
       >>> split_name('A RICHARD A', ['II', 'III', 'IV', 'JR', 'SR'], '')
       ['ARICHARD', 'A', '', '', 'A RICHARD']
       >>> split_name('J EDGAR V E JR', ['II', 'III', 'IV', 'JR', 'SR'])
       ['JEDGAR', 'V', 'E', 'JR', 'J EDGAR']
       >>> split_name('GEORGE H W', ['II', 'III', 'IV', 'JR', 'SR'])
       ['GEORGE', 'H', 'W', '', 'GEORGE']
    '''
    # Extract middle initial
    x, mi = extract_middle_initial(x, mi_pattern, not_pattern, suffixes)
    # Extract second middle initial
    x, mi2 = extract_middle_initial(x, mi_pattern, not_pattern, suffixes)
    # Extract suffix name
    x, suff = extract_suffix_name(x, suffixes)
    # If first initial is empty then fill it with first letter of x
    # Return list of fully stripped name (no spaces), middle initial,
    # middle initial 2, suffix, and basic stripped name
    out_list = [full_strip_name(x).replace(' ', ''),
                mi, mi2, suff,
                x]
    return out_list
doctest.run_docstring_examples(split_name, globals())

In [261]:
def clean_name_col(col):
    '''returns pandas dataframe of name columns
       PYTEST
    '''
    # Store name of input column
    col_name = col.name
    # Remove redundant spaces and periods
    col = col.map(basic_strip_name)
    # Ensure string is in uppercase
    col = col.map(str.upper)

    # If col is first or last name
    if col_name in ['first_name', 'last_name']:
        # Assign middle_initial pattern to be:
        # single letters between spaces or at end of string
        mi_pattern='(\s[A-Z]\s)|(\s[A-Z]$)'
        # Choose regex pattern to avoid in middle initial selection:
        # If first name: no not_pattern
        # If last name: avoid DE LA _ pattern (or others added as needed)
        not_pattern = ('' if col_name == 'first_name'
                      else '^((DE LA)\s[A-Z]($|\s))')
        # Chooses suffixes to look for depending on column:
        # First Name columns probably do no contain V as a suffix
        # V may be mistaken as a middle initial
        # But Last Name columns might contain V or I as a suffix
        suffixes = ['II', 'III', 'IV', 'JR', 'SR']
        if col_name == 'last_name':
            suffixes.extend(['V', 'I'])
        # Return pandas dataframe created from list of lists
        # Comprised of first/last name (with no separation - NS),
        # middle initial, middle inital 2,
        # suffix name, and first/last name (with spaces or dashes).
        # Middle initial and suffix name elements are abbrivated
        # and which column they came from is idenified by F/L
        return pd.DataFrame([split_name(x, suffixes,
                                        not_pattern,
                                        mi_pattern)
                             for x in col],
                            columns=[col_name + '_NS',
                                     col_name[0] + '_MI',
                                     col_name[0] + '_MI2',
                                     col_name[0] + '_SN',
                                     col_name])

    # Else if column is a middle name column
    elif col_name == 'middle_name':
        # Return pandas dataframe of middle name
        # And first letter of middle name (middle initial)
        # If there is not middle name, return empty strings
        return pd.DataFrame([['', '']
                             if not (isinstance(x, str) and x)
                             else [x, x[0]]
                             for x in col],
                            columns=[col_name, 'middle_initial'])

    # If column is middle initial
    elif col_name == 'middle_initial':
        # Return middle initial column
        return col

    # If column not recognized, raise error
    else:
        raise Exception('Uh.. what sort of names?')

In [265]:
df_cols = []
fn_df = clean_name_col(names['first_name'])
# Split last name column into last name, middle initial, suffix
ln_df = clean_name_col(names['last_name'])
# Join name dataframes
cleaned_df = fn_df.join(ln_df)
# If suffix not in columns already (almost always)
if 'suffix_name' not in df_cols:
    # Insert suffix name column of empty strings
    cleaned_df.insert(0, 'suffix_name', '')

# If middle name is already in columns
if 'middle_name' in df_cols:
    # Join cleaned middle name and initial columns to cleaned_df
    cleaned_df = cleaned_df.join(clean_name_col(name_df['middle_name']))
# If middle initial is already in columns
elif 'middle_initial' in df_cols:
    # Join cleaned middle initial column to cleaned_df
    cleaned_df = cleaned_df.join(clean_name_col(name_df['middle_initial']))
# If no middle inital columns in original dataframe
else:
    # Insert middle initial column of empty strings
    cleaned_df.insert(0, 'middle_initial', '')
    
cleaned_df

,middle_initial,suffix_name,first_name_NS,f_MI,f_MI2,f_SN,first_name,last_name_NS,l_MI,l_MI2,l_SN,last_name
0,,,DEANDRE,,,,DE ANDRE,DELAO,,,,DE LA O
1,,,BOB,,,,BOB,JONES,,,V,JONES
2,,,ARICHARD,A,,,A RICHARD,LUQUEROSALES,,,,LUQUE-ROSALES
3,,,KIMTOY,,,,KIM-TOY,MCCARTHY,,,,MC CARTHY
4,,,JOANN,,,,JO ANN,OBRIEN,,,,O BRIEN
5,,,JT,,,,J T,SADOWSKY,,,JR,SADOWSKY
6,,,ABDULAZIZ,,,,ABDUL-AZIZ,NORWOOD,,,II,NORWOOD
7,,,MARCELLUS,,,,MARCELLUS,BURKE,J,,JR,BURKE
8,,,ERIN,E,M,,ERIN,VONKONDRAT,,,,VON KONDRAT
9,,,JEDGAR,,,,J EDGAR,HOOVER,,,,HOOVER


In [283]:
compare_columns(cleaned_df[['middle_initial', 'f_MI', 'f_MI2', 'l_MI', 'l_MI2']],
               2, ['middle_initial1', 'middle_initial2'])

,middle_initial1,middle_initial2
0,,
1,,
2,A,
3,,
4,,
5,,
6,,
7,J,
8,E,M
9,,


In [284]:
compare_columns(cleaned_df[['suffix_name', 'f_SN', 'l_SN']],
               1, ['suffix_name'])

,suffix_name
0,
1,V
2,
3,
4,
5,JR
6,II
7,JR
8,
9,


In [401]:
def compare_columns(df, colnames):
    '''
    '''
    output_list = []
    ncols = len(colnames)
    for i, row in df.iterrows():
        x = list(filter(bool, row))
        assert len(x) <= ncols, 'too many {}'.format(x)
        output_list.append(x)
    return pd.DataFrame(output_list, columns = colnames)

def clean_names(df):
    '''returns pandas dataframe of cleaned name columns
       PYTEST
    '''
    df_cols = list(df.columns)  # Store column names
    df_rows = df.shape[0] # Store full df shape
    # Insert df index as column in case of shuffling
    df.insert(0, 'Index', df.index)
    # Create name_df as unique rows in input df
    name_df = (df[df_cols].drop_duplicates()
                 .copy()
                 .reset_index(drop=True))
    # Give names an ID by index
    name_df.insert(0, 'ID', name_df.index)
    # Give ID to full data
    df = df.merge(name_df, on=df_cols, how='inner')
    # Ensure no rows were dropped
    assert df.shape[0] == df_rows,\
    'Lost rows when giving ID: {0} vs. {1}'.format(df_rows,
                                                   df.shape[0])
    # Then drop all columns except for ID
    df.drop(df_cols, axis=1, inplace=True)
    
    
    # If names are full names
    if 'full_name' in df_cols:
        # Split full names into last and first names
        name_df = split_full_names(name_df['full_name'])
    # Otherwise (generally first and last names)
    else:
        name_df = name_df.fillna("")  # Fill NAs with empty strings

    # Split first name column into first name, middle initial, suffix
    ## natural language processing for name parsing english
    fn_df = clean_name_col(name_df['first_name'])
    # Split last name column into last name, middle initial, suffix
    ln_df = clean_name_col(name_df['last_name'])
    # Join name dataframes
    cleaned_df = fn_df.join(ln_df)

    # If suffix not in columns already (almost always)
    if 'suffix_name' not in df_cols:
        # Insert suffix name column of empty strings
        cleaned_df.insert(0, 'suffix_name', '')

    # If middle name is already in columns
    if 'middle_name' in df_cols:
        # Join cleaned middle name and initial columns to cleaned_df
        cleaned_df = cleaned_df.join(clean_name_col(name_df['middle_name']))
    # If middle initial is already in columns
    elif 'middle_initial' in df_cols:
        # Join cleaned middle initial column to cleaned_df
        cleaned_df = cleaned_df.join(clean_name_col(name_df['middle_initial']))
    # If no middle inital columns in original dataframe
    else:
        # Insert middle initial column of empty strings
        cleaned_df.insert(0, 'middle_initial', '')
    # Create middle initial dataframe for comparing
    mi_df = cleaned_df[['middle_initial', 'f_MI', 'f_MI2', 'l_MI', 'l_MI2']]
    mi_df = compare_columns(mi_df,
                            colnames=['middle_initial', 'middle_initial2'])
    
    # Create suffix name dataframe (suffix_name)
    sn_df = cleaned_df[['suffix_name', 'f_SN', 'l_SN']]
    sn_df = compare_columns(sn_df,
                            colnames=['suffix_name'])

    # Identify name columns which exist in cleaned_df
    name_cols = intersect(cleaned_df.columns,
                          ['first_name', 'last_name',
                           'first_name_NS', 'last_name_NS',
                           'middle_name'])
    # Remove middle initial and suffix columns from cleaned dataframe
    # Then joined cleaned dataframe to middle initial and suffix name dfs
    cleaned_df = cleaned_df[name_cols].join(mi_df).join(sn_df)
    cleaned_df.fillna('', inplace=True)
    
    # Expand cleaned_df to size of original df
    # by giving ID column (index)
    # then merging to df on ID and delete ID
    cleaned_df.insert(0, 'ID', cleaned_df.index)
    cleaned_df = cleaned_df.merge(df, on='ID', how='inner')
    del cleaned_df['ID']
    # Ensure row numbers are correct
    assert df.shape[0] == df_rows,\
    ('Lost rows when remerging: {0} vs. {1}'
     '').format(cleaned_df.shape[0],
                df.shape[0])
    # Sort dataframe by original index
    # and delete index column
    cleaned_df = (cleaned_df
                    .sort_values('Index')
                    .reset_index(drop=True)
                    .drop('Index', axis=1))
    # Return cleaned names dataframe
    return cleaned_df


In [23]:
out_names = pd.DataFrame({'first_name' : ['DE ANDRE', 'BOB','RICHARD', 'KIM-TOY', 'JO ANN', 'J T', 'ABDUL-AZIZ', 'MARCELLUS', 'ERIN', 'EDGAR', 'GEORGE'],
                          'middle_name1': ['','D', 'A', '', '', '', '', 'H', 'E','','H'],
                          'middle_name2': ['','', '', '', '', '', '', 'J', 'M','','W'],
                          'first_name_initial': ['', '', 'A', '', '', '', '','', '', 'J', ''],
                         'suffix_name': ['', 'V', '', '','', 'JR', 'II', 'JR', '', '', ''],
                         'last_name': ['DE LA O', 'JONES', 'LUQUE-ROSALES', 'MC CARTHY', 'O BRIEN', 'SADOWSKY', 'NORWOOD', 'BURKE', 'VON KONDRAT', 'HOOVER', 'BUSH'],
                         'first_name_NS': ['DEANDRE', 'BOB','RICHARD', 'KIMTOY', 'JOANN', 'JT', 'ABDULAZIZ', 'MARCELLUS', 'ERIN', 'EDGAR', 'GEORGE'],
                         'last_name_NS':['DELAO', 'JONES', 'LUQUEROSALES', 'MCCARTHY', 'OBRIEN', 'SADOWSKY', 'NORWOOD', 'BURKE', 'VONKONDRAT', 'HOOVER', 'BUSH']})
                         
                         
                     #'last_name' : ['DE LA O','JONES V', 'LUQUE-ROSALES', 'MC CARTHY', 'O BRIEN', 'SADOWSKY, JR', 'NORWOOD II', 'BURKE JR J', 'ERIN  E M', 'HOOVER', 'BUSH']})

In [260]:
out_names

,first_name,first_name_NS,first_name_initial,last_name,last_name_NS,middle_name1,middle_name2,suffix_name
0,DE ANDRE,DEANDRE,,DE LA O,DELAO,,,
1,BOB,BOB,,JONES,JONES,D,,V
2,RICHARD,RICHARD,A,LUQUE-ROSALES,LUQUEROSALES,A,,
3,KIM-TOY,KIMTOY,,MC CARTHY,MCCARTHY,,,
4,JO ANN,JOANN,,O BRIEN,OBRIEN,,,
5,J T,JT,,SADOWSKY,SADOWSKY,,,JR
6,ABDUL-AZIZ,ABDULAZIZ,,NORWOOD,NORWOOD,,,II
7,MARCELLUS,MARCELLUS,,BURKE,BURKE,H,J,JR
8,ERIN,ERIN,,VON KONDRAT,VONKONDRAT,E,M,
9,EDGAR,EDGAR,J,HOOVER,HOOVER,,,


In [374]:
awd = pd.read_csv('awards.csv.gz')[['last_name', 'first_name', 'middle_initial']]
acc = pd.read_csv('accused.csv.gz')[['full_name']]
ase = pd.read_csv('ase-units.csv.gz')[['last_name', 'first_name']]

/Users/invinst/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/invinst/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [375]:
%%time
awd = clean_names(awd)

CPU times: user 6.4 s, sys: 240 ms, total: 6.64 s
Wall time: 6.8 s


In [376]:
%%time
acc = clean_names(acc)

CPU times: user 2.71 s, sys: 26.7 ms, total: 2.74 s
Wall time: 2.75 s


In [377]:
ase = pd.read_csv('ase-units.csv.gz')[['last_name', 'first_name']]
%lprun -f clean_names ase = clean_names(ase)

In [396]:
trro = pd.read_csv('trr-officers.csv.gz')

In [397]:
%%time
trro = clean_names(trro[['first_name', 'last_name']])

CPU times: user 2.09 s, sys: 30.8 ms, total: 2.12 s
Wall time: 2.17 s


In [398]:
trro

,first_name_NS,last_name,last_name_NS,first_name,middle_initial,middle_initial2,suffix_name,Index
0,ANGELO,VELAZQUEZ,VELAZQUEZ,ANGELO,,,,0
1,ANDRE,REYES,REYES,ANDRE,,,,1
2,MARIO,LIMON,LIMON,MARIO,,,,2
3,ANTONIO,HERRERA,HERRERA,ANTONIO,,,,3
4,DEJAN,MILOVANOVIC,MILOVANOVIC,DEJAN,,,,4
5,SALVATORE,LAZZARA,LAZZARA,SALVATORE,,,JR,5
6,HERBERT,BETANCOURT,BETANCOURT,HERBERT,,,,6
7,JESUS,ESPINOSA,ESPINOSA,JESUS,,,,7
8,BRIAN,MCHALE,MCHALE,BRIAN,,,,8
9,FROILAN,BUCIO,BUCIO,FROILAN,,,JR,9


In [399]:
trroo = pd.read_csv('trr-officers.csv.gz')

In [400]:
trroo[['first_name', 'last_name']]

,first_name,last_name
0,ANGELO,VELAZQUEZ
1,ANDRE,REYES
2,MARIO,LIMON
3,ANTONIO,HERRERA
4,DEJAN,MILOVANOVIC
5,SALVATORE,LAZZARA JR
6,HERBERT,BETANCOURT
7,JESUS,ESPINOSA
8,BRIAN,MCHALE
9,FROILAN,BUCIO JR


In [402]:
names2 = clean_names(names)

In [404]:
names

,Index,first_name,last_name
0,0,DE ANDRE,DE LA O
1,1,BOB D,JONES V
2,2,A RICHARD A,LUQUE-ROSALES
3,3,KIM-TOY,MC CARTHY
4,4,JO ANN,O BRIEN
5,5,J T,"SADOWSKY, JR"
6,6,ABDUL-AZIZ,NORWOOD II
7,7,MARCELLUS. H,BURKE JR J
8,8,ERIN E M,VON KONDRAT
9,9,J EDGAR,HOOVER


In [403]:
names2

,first_name_NS,last_name,last_name_NS,first_name,middle_initial,middle_initial2,suffix_name
0,DEANDRE,DE LA O,DELAO,DE ANDRE,,,
1,BOB,JONES,JONES,BOB,D,,V
2,ARICHARD,LUQUE-ROSALES,LUQUEROSALES,A RICHARD,A,,
3,KIMTOY,MC CARTHY,MCCARTHY,KIM-TOY,,,
4,JOANN,O BRIEN,OBRIEN,JO ANN,,,
5,JT,SADOWSKY,SADOWSKY,J T,,,JR
6,ABDULAZIZ,NORWOOD,NORWOOD,ABDUL-AZIZ,,,II
7,MARCELLUS,BURKE,BURKE,MARCELLUS,H,J,JR
8,ERIN,VON KONDRAT,VONKONDRAT,ERIN,E,M,
9,JEDGAR,HOOVER,HOOVER,J EDGAR,,,


In [380]:
out_names

,first_name,first_name_NS,first_name_initial,last_name,last_name_NS,middle_name1,middle_name2,suffix_name
0,DE ANDRE,DEANDRE,,DE LA O,DELAO,,,
1,BOB,BOB,,JONES,JONES,D,,V
2,RICHARD,RICHARD,A,LUQUE-ROSALES,LUQUEROSALES,A,,
3,KIM-TOY,KIMTOY,,MC CARTHY,MCCARTHY,,,
4,JO ANN,JOANN,,O BRIEN,OBRIEN,,,
5,J T,JT,,SADOWSKY,SADOWSKY,,,JR
6,ABDUL-AZIZ,ABDULAZIZ,,NORWOOD,NORWOOD,,,II
7,MARCELLUS,MARCELLUS,,BURKE,BURKE,H,J,JR
8,ERIN,ERIN,,VON KONDRAT,VONKONDRAT,E,M,
9,EDGAR,EDGAR,J,HOOVER,HOOVER,,,
